In [1]:
import QuantLib as ql
import prettytable as pt

In [2]:
coupon, maturityDate, issueDate = (0.90, ql.Date(1, 4, 2031), ql.Date(1, 10, 2020))
eval_date = ql.Date(1, 4, 2022)
base_yld = 2.0
base_prc = 91.04

In [3]:
ql.Settings.instance().evaluationDate = ql.Date(1, 4, 2022)

In [4]:
bondYield = 2.0 / 100.0

In [25]:
settlementDays = 2
faceAmount = 100.0

tenor = ql.Period(ql.Semiannual)
calendar = ql.Italy()
convention = ql.Unadjusted
terminationDateConvention = convention
rule = ql.DateGeneration.Backward
endOfMonth = False

schedule = ql.Schedule(issueDate, maturityDate, tenor, calendar, convention, terminationDateConvention, rule, endOfMonth)
coupons = ql.DoubleVector(1)
coupons[0] = coupon / 100.0

accrualDayCounter = ql.ActualActual(ql.ActualActual.Bond, schedule)
paymentConvention = ql.Unadjusted
bond = ql.FixedRateBond(settlementDays, faceAmount, schedule, coupons, accrualDayCounter, paymentConvention)

curve = ql.FlatForward(eval_date, 
                       ql.QuoteHandle(ql.SimpleQuote(bondYield)), 
                       accrualDayCounter, 
                       ql.Compounded)
handle = ql.YieldTermStructureHandle(curve)
handle.enableExtrapolation()
bondEngine = ql.DiscountingBondEngine(handle)    
bond.setPricingEngine(bondEngine)

cleanPrice = bond.cleanPrice()
dirtyPrice = bond.dirtyPrice()
accruedAmount = bond.accruedAmount()

tab = pt.PrettyTable(['item', 'QuantLib', 'Expected'])
tab.add_row(['clean price', cleanPrice, base_prc])
tab.add_row(['dirty price', dirtyPrice, base_prc+0.0098])
tab.add_row(['accrued amount', accruedAmount, 0.0098])
tab.float_format = '.4'

print(tab)

+----------------+----------+----------+
|      item      | QuantLib | Expected |
+----------------+----------+----------+
|  clean price   | 91.0680  | 91.0400  |
|  dirty price   | 91.0778  | 91.0498  |
| accrued amount |  0.0098  |  0.0098  |
+----------------+----------+----------+
